#**Importing necessary libraries**


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

#**Importing dataset**


In [2]:
df = pd.read_csv("megaGymDataset.csv")
df.head()

,Unnamed: 0,Title,Desc,Type,BodyPart,Equipment,Level,Rating,RatingDesc
0,0,Partner plank band row,The partner plank band row is an abdominal exe...,Strength,Abdominals,Bands,Intermediate,0.0,NaN
1,1,Banded crunch isometric hold,The banded crunch isometric hold is an exercis...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
2,2,FYR Banded Plank Jack,The banded plank jack is a variation on the pl...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
3,3,Banded crunch,The banded crunch is an exercise targeting the...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
4,4,Crunch,The crunch is a popular core exercise targetin...,Strength,Abdominals,Bands,Intermediate,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2918 entries, 0 to 2917
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  2918 non-null   int64  
 1   Title       2918 non-null   object 
 2   Desc        1368 non-null   object 
 3   Type        2918 non-null   object 
 4   BodyPart    2918 non-null   object 
 5   Equipment   2918 non-null   object 
 6   Level       2918 non-null   object 
 7   Rating      1031 non-null   float64
 8   RatingDesc  862 non-null    object 
dtypes: float64(1), int64(1), object(7)
memory usage: 205.3+ KB


In [4]:
df.describe()

,Unnamed: 0,Rating
count,2918.000000,1031.000000
mean,1458.500000,5.919690
std,842.498368,3.584607
min,0.000000,0.000000
25%,729.250000,3.000000
50%,1458.500000,7.900000
75%,2187.750000,8.700000
max,2917.000000,9.600000


In [5]:
df.columns = df.columns.str.replace('Unnamed: 0', 'index')


#**Getting input from user**

In [6]:
level = input("What is your level?")
days = int(input("How many days per week can you go to the gym?"))
goal = input("What is your goal ?") #Cut/Bulk/Lean/Strength/Athletic


What is your level?Beginner
How many days per week can you go to the gym?4
What is your goal ?Cut


In [7]:
home = input("Do you prefer to workout at home?")
if (home=="Yes") :
  equip=input("what equipment do you have?")
  equipments = equip.split(",")
  equipments.append("NaN")
  print(equipments)

Do you prefer to workout at home?No


In [8]:
programs={}
Push = [{'Triceps' : 2}, {'Shoulders' : 2} , {'Chest' :3} ]
Pull = [{'Biceps' :2}, {'Forearms' :1}, {'Lats' :1}, {'Middle Back':1}, {'Traps':1}]
Legs = [ {'Abductors' :1} , {'Hamstrings' :2}, {'Quadriceps':2}]
Glutes = [{'Adductors':1}, {'Glutes':1}]
Core = [{'Abdominals' :3}, {'Lower Back':2}]

Upper = Push + Pull
Lower = Legs + Glutes


#**Choosing program based on available days and goal**

In [9]:
if days==2:
    programs.update({'Upper': Upper})
    programs.update({'Legs' : Legs})
elif days == 3:
    programs['Push'] = Push
    programs['Pull'] = Pull
    programs['Legs'] = Legs
elif days==4:
    programs['Push'] = Push
    programs['Pull'] = Pull
    programs['Legs'] = Legs
    programs['Core'] = Core
elif days == 5:
    programs['Chest'] = Push + ['Chest']
    programs['Back'] = Pull + ['Middle Back']
    programs['Shoulder'] = Push + ['Shoulders']
    programs['Legs'] = Legs
    programs['Core'] = Core
print(programs)


{'Push': [{'Triceps': 2}, {'Shoulders': 2}, {'Chest': 3}], 'Pull': [{'Biceps': 2}, {'Forearms': 1}, {'Lats': 1}, {'Middle Back': 1}, {'Traps': 1}], 'Legs': [{'Abductors': 1}, {'Hamstrings': 2}, {'Quadriceps': 2}], 'Core': [{'Abdominals': 3}, {'Lower Back': 2}]}


In [10]:
Types=[]
if (goal=="Cut") :
  Types=[{'Cardio' : 0.5},{'Strength' : 0.5}]
elif(goal=="Lean") :
  Types=[{'Cardio' : 0.25},{'Strength' : 0.75}]
elif(goal=="Bulk") :
  Types=[{'Powerlifting' : 0.5},{'Strength' : 0.5}]
elif(goal=="Strength") :
  Types=[{'Strongman' : 0.5},{'Strength' : 0.5}]
elif(goal=="Athletic") :
  Types=[{'Plyometrics' : 0.5},{'Olympic Weightlifting' : 0.5}]

In [11]:
key1=list(Types[0].keys())[0]
key2=list(Types[1].keys())[0]
coeff1=Types[0][key1]
coeff2=Types[1][key2]

#**Affecting program exercises**

In [15]:
from IPython.display import display
selected_program = pd.DataFrame(columns=['Day','BodyPart', 'Type', 'SelectedExercises', 'Equipement'])
if(home=="No"):
  for day in programs:
      for part in programs[day]:
        filtered_exercises = df[(df['BodyPart'] == next(iter(part))) & (df['Type'] == key1)]
        top_exercises = filtered_exercises.sort_values(by='Rating', ascending=False).head(round(part[next(iter(part))]*coeff1))
        exercise_titles = top_exercises['Title'].tolist()
        count_exos=len(top_exercises)
        if(count_exos>=0):
          selected_program = pd.concat([selected_program, pd.DataFrame({'Day' : [day] , 'BodyPart': [next(iter(part))], 'Type' : key1 ,'SelectedExercises': [exercise_titles]})], ignore_index=True)

          filtered_exercises_S = df[(df['BodyPart'] == next(iter(part))) & (df['Type'] == key2) & (df['Level'] == level)]
          top_exercises_S = filtered_exercises_S.sort_values(by='Rating', ascending=False).head(part[next(iter(part))]-count_exos)
          exercise_titles_S=top_exercises_S['Title'].tolist()
        selected_program = pd.concat([selected_program, pd.DataFrame({'Day' : [day] , 'BodyPart': [next(iter(part))], 'Type' : key2, 'SelectedExercises': [exercise_titles_S]})], ignore_index=True)
else :
  for day in programs:
      for part in programs[day]:
        filtered_exercises = df[(df['BodyPart'] == next(iter(part))) & (df['Type'] == key1) & df['Equipment'].isin(equipments)]
        top_exercises = filtered_exercises.sort_values(by='Rating', ascending=False).head(round(part[next(iter(part))]*coeff1))
        exercise_titles = top_exercises['Title'].tolist()
        count_exos=len(top_exercises)
        if(count_exos>=0):
          selected_program = pd.concat([selected_program, pd.DataFrame({'Day' : [day] , 'BodyPart': [next(iter(part))], 'Type' : key1 ,'SelectedExercises': [exercise_titles]})], ignore_index=True)

          filtered_exercises_S = df[(df['BodyPart'] == next(iter(part))) & (df['Type'] == key2) & (df['Level'] == level)]
          top_exercises_S = filtered_exercises_S.sort_values(by='Rating', ascending=False).head(part[next(iter(part))]-count_exos)
          exercise_titles_S=top_exercises_S['Title'].tolist()
        selected_program = pd.concat([selected_program, pd.DataFrame({'Day' : [day] , 'BodyPart': [next(iter(part))], 'Type' : key2, 'SelectedExercises': [exercise_titles_S]})], ignore_index=True)
display(selected_program)


,Day,BodyPart,Type,SelectedExercises,Equipement
0,Push,Triceps,Cardio,[],NaN
1,Push,Triceps,Strength,"[Decline Close-Grip Bench To Skull Crusher, De...",NaN
2,Push,Shoulders,Cardio,[Shadow boxing],NaN
3,Push,Shoulders,Strength,[Power Partials],NaN
4,Push,Chest,Cardio,[Swimming],NaN
5,Push,Chest,Strength,"[Bodyweight Flyes, Reverse-grip incline dumbbe...",NaN
6,Pull,Biceps,Cardio,[],NaN
7,Pull,Biceps,Strength,"[Incline Hammer Curls, Wide-grip barbell curl]",NaN
8,Pull,Forearms,Cardio,[],NaN
9,Pull,Forearms,Strength,[Standing behind-the-back wrist curl],NaN
